# Week 03 Assignment Covid

Welcome to **week three** of this course programming 1. You will learn about combining data with pandas and numpy and you will learn to visualize with bokeh. Concretely, you will preprocess the partly Synthetic Covid data in an appropiate format in order to conduct statistical and visual analysis. Learning outcomes:

- Load a tabular dataset
- Inspect the dataset for quality and metadata information
- Combine data from several tables into one dataframe
- Subselect specific data from dataframes
- Reshape the dataset into a format suitable for visual and statistical analysis
- Visualize data using bokeh 
- Use widgets to make the plot interactive (optional)
- Use geomap to plot locations (optional)


Your job is to **visualize the lab values taken for COVID-19 patients of survived versus not survived patients**. 

The assignment consists of 6 parts:

- [part 1: load the data](#0)
     - [Exercise 1.1](#ex-11)
- [part 2: data wrangling](#1)
     - [Exercise 2.1](#ex-21)
- [part 3: more wrangling](#2)
     - [Exercise 3.1](#ex-31)
- [part 4: plot the data](#3)
     - [Exercise 4.1](#ex-41)
- [part 5: plot patient location](#5)
     - [Exercise 5.1](#ex-51)


Part 1 and 4 are mandatory, part 5 is optional (bonus)
To pass the assingnment you need to a score of 60%. 


## About the data

The data is generated by Synthea's COVID-19 module. The data was constructed using three peer-reviewed publications published in the early stages of the global pandemic, when less was known, along with emerging resources, data, publications, and clinical knowledge. The simulation outputs synthetic Electronic Health Records (EHR), including the daily consumption of Personal Protective Equipment (PPE) and other medical devices and supplies. For this assignment the `conditions`, `patients`, `observations`, `careplans` and `encounters` table will be used. The Data is stored in separate tables to avoid redundancy, with as a concequence that tables need to be combined and reorganized in dataframes for analysing purpose.

Source: Walonoski J, Klaus S, Granger E, Hall D, Gregorowicz A, Neyarapally G, Watson A, Eastman J. Synthea™ Novel coronavirus (COVID-19) model and synthetic data set. Intelligence-Based Medicine. 2020 Nov;1:100007. https://doi.org/10.1016/j.ibmed.2020.100007

Please <a href = "https://storage.googleapis.com/synthea-public/10k_synthea_covid19_csv.zip">download</a> the data

#### Covid Patients
Patients are considered Covid patients if they are identified with `CODE` `840539006`


#### Survivors
Patients that had covid and where tested negative after isolation have tested code `94531-1`,  SARS-CoV-2 RNA Pnl Resp NAA+probe (covid-sars test) + a value of `Not detected (qualifier value)`. These patients are considered to be survived covid patients. 

#### Non-Survivors
Patients that did not survived Covid have a `DEATHDATE` which is not null. 


#### Lab values  COVID-19 patients

Patients are monitored for blood and heart conditions once they are admitted in Hospital or under treatment. The lab values of interest are as follow: 

- `48065-7`  Fibrin D-dimer FEU [Mass/volume] in Platelet poor plasma
- `26881-3`   Interleukin 6 [Mass/volume] in Serum or Plasma
- `2276-4` Ferritin [Mass/volume] in Serum or Plasma
- `89579-7` Troponin I.cardiac [Mass/volume] in Serum or Plasma by High sensitivity method
- `731-0` Lymphocytes [#/volume] in Blood by Automated count
- `14804-9` Lactate dehydrogenase [Enzymatic activity/volume] in Serum or Plasma by Lactate to pyruvate reaction


---

<a name='0'></a>
## Part 1: Load the data (20 pt)

Instructions: Load the data of the following files

- conditions.csv
- patients.csv
- observations.csv
- careplans.csv
- encounters.csv

Get yourself familiar with the data. Create some meaningful overviews. Answer the following questions

1. How many patients are there
2. How many covid-patients are there
3. How many patients do have a 'Hospital admission for isolation' encounter
    
<details>    
<summary>
    <font size="3" color="darkgreen"><b>Hints</b></font>
</summary>
    <ul><li>use a unique dataframe for each file, use a meaningful name</li>
    <li>pandas.read_csv() method can be used to read a csv file</li>
    <li>pandas.DataFrame.head() method is often used to inspect the dataframe</li>
    <li>.unique() returns a list of unique values of a column</li>
</ul>
</details>

<a name='ex-11'></a>
### 1.1 Code your solution

-----------

Amount of patients can be derived from the `SSN` columns, as the social security number is unique for every citizen. Therefore, there are  patients.

-----------

In [1]:
import pandas as pd
import numpy as np

In [2]:
base_path = '../data/covid/'

conditions = pd.read_csv(base_path + 'conditions.csv')
patients = pd.read_csv(base_path + 'patients.csv')
observations = pd.read_csv(base_path + 'observations.csv')
careplans = pd.read_csv(base_path + 'careplans.csv')
encounters = pd.read_csv(base_path + 'encounters.csv')

datasets = {
    'conditions': conditions,
    'patients': patients,
    'observations': observations,
    'careplans': careplans,
    'encounters': encounters
}

In [3]:
from IPython.display import display_html
from itertools import chain,cycle

def display_side_by_side(*args,titles=cycle([''])):
    """https://stackoverflow.com/questions/38783027/jupyter-notebook-display-two-pandas-tables-side-by-side"""
    html_str=''
    for df,title in zip(args, chain(titles,cycle(['</br>'])) ):
        html_str+='<th style="text-align:center"><td style="vertical-align:top">'
        html_str+=f'<h2>{title}</h2>'
        html_str+=df.to_html().replace('table','table style="display:inline"')
        html_str+='</td></th>'
    display_html(html_str,raw=True)

--------------

**Finding the number of patients**

Normally we can assume the SSN is unique.
"Your nine-digit Social Security number is your first and continuous connection with Social Security. It helps us identify and accurately record your covered wages or self-employment earnings. We also use it to monitor your record once you start getting benefits." 
- https://www.ssa.gov/ssnumber/

In this case, the SSN number is not unique, as there are several duplicated values. It is not clear why SSN is not unique. It could be because of typing mistakes or other factors.

A column that looks unique, is the `Id` column. However, the `Id` column has a downside, because the entries can correspond to database entries. Thus, it does not represent the number of patients.

To retrieve the number of patients, the better way is to use different columns as identifiers. As the `SSN` is not unique, the `BIRTHDATE` column is used as an identifier as well, because there are no missing values. In this way, all unique patients should be found. 

NOTE
- Because the `SSN` and `BIRTHDAY` column give unique patients, the Id column can also be used, as every row is a unique patient.


In [4]:
patients.head()

,Id,BIRTHDATE,DEATHDATE,SSN,DRIVERS,PASSPORT,PREFIX,FIRST,LAST,SUFFIX,...,BIRTHPLACE,ADDRESS,CITY,STATE,COUNTY,ZIP,LAT,LON,HEALTHCARE_EXPENSES,HEALTHCARE_COVERAGE
0,f0f3bc8d-ef38-49ce-a2bd-dfdda982b271,2017-08-24,NaN,999-68-6630,NaN,NaN,NaN,Jacinto644,Kris249,NaN,...,Beverly Massachusetts US,888 Hickle Ferry Suite 38,Springfield,Massachusetts,Hampden County,1106.0,42.151961,-72.598959,8446.49,1499.08
1,067318a4-db8f-447f-8b6e-f2f61e9baaa5,2016-08-01,NaN,999-15-5895,NaN,NaN,NaN,Alva958,Krajcik437,NaN,...,Boston Massachusetts US,1048 Skiles Trailer,Walpole,Massachusetts,Norfolk County,2081.0,42.177370,-71.281353,89893.40,1845.72
2,ae9efba3-ddc4-43f9-a781-f72019388548,1992-06-30,NaN,999-27-3385,S99971451,X53218815X,Mr.,Jayson808,Fadel536,NaN,...,Springfield Massachusetts US,1056 Harris Lane Suite 70,Chicopee,Massachusetts,Hampden County,1020.0,42.181642,-72.608842,577445.86,3528.84
3,199c586f-af16-4091-9998-ee4cfc02ee7a,2004-01-09,NaN,999-73-2461,S99956432,NaN,NaN,Jimmie93,Harris789,NaN,...,Worcester Massachusetts US,201 Mitchell Lodge Unit 67,Pembroke,Massachusetts,Plymouth County,NaN,42.075292,-70.757035,336701.72,2705.64
4,353016ea-a0ff-4154-85bb-1cf8b6cedf20,1996-11-15,NaN,999-60-7372,S99917327,X58903159X,Mr.,Gregorio366,Auer97,NaN,...,Patras Achaea GR,1050 Lindgren Extension Apt 38,Boston,Massachusetts,Suffolk County,2135.0,42.352434,-71.028610,484076.34,3043.04


In [5]:
# Somehow SSN contains duplicated values
patients.duplicated(subset=['SSN'], keep=False).sum()

195

In [6]:
# Inspect the duplicated values and find out how to get unique number of patients.
patients[patients['SSN'].duplicated(keep=False)].sort_values(by='SSN').head()

,Id,BIRTHDATE,DEATHDATE,SSN,DRIVERS,PASSPORT,PREFIX,FIRST,LAST,SUFFIX,...,BIRTHPLACE,ADDRESS,CITY,STATE,COUNTY,ZIP,LAT,LON,HEALTHCARE_EXPENSES,HEALTHCARE_COVERAGE
12113,ee239318-69ec-4ca1-b0c6-28de5127bff5,1920-06-20,1973-06-15,999-10-5624,S99943358,X30277910X,Mrs.,Beatriz277,Saldivar860,NaN,...,Santo Domingo National District DO,623 Beer View,Springfield,Massachusetts,Hampden County,1104.0,42.110942,-72.623600,195781.57,106891.65
1216,951ccbdf-887a-4e8b-b9a0-2c3b40477f8c,2007-09-12,2017-10-04,999-10-5624,NaN,NaN,NaN,Luetta803,Welch179,NaN,...,East Longmeadow Massachusetts US,481 Cruickshank Union,Brockton,Massachusetts,Plymouth County,2351.0,42.135237,-70.978394,271401.65,2454.04
6662,3f923231-b012-4fac-9c46-0a345caf491a,2012-04-10,NaN,999-11-5878,NaN,NaN,NaN,Wilhemina814,Schaden604,NaN,...,Plainville Massachusetts US,545 Bartoletti Gardens,Springfield,Massachusetts,Hampden County,1199.0,42.095911,-72.558645,218614.73,2195.72
10593,a79b1e70-74a4-4f9e-9a9b-63e2e988a28a,1987-03-10,NaN,999-11-5878,S99944870,X43998566X,Ms.,Sharita792,Reichert620,NaN,...,Topsfield Massachusetts US,596 Grady Dam,Boston,Massachusetts,Suffolk County,2132.0,42.364210,-71.130013,55076.31,3204.72
11838,ce6c0b96-d708-45a0-9b56-720e62a462aa,1992-06-21,NaN,999-12-7314,S99987345,X74728398X,Mr.,Francis500,Moore224,NaN,...,West Springfield Massachusetts US,480 O'Connell Drive Unit 86,Templeton,Massachusetts,Worcester County,NaN,42.578473,-72.116841,9612.25,0.00


In [7]:
# Check if the BIRTHDATE columns have no missing values
missing = pd.to_datetime(patients['BIRTHDATE']).isna().sum()

if missing == 0:
    print('There are no missing birthday values.')
else:
    print('There are some missing birthday values. Investigate.')
    
# Check if having an identifier based on SSN and BIRTHDATE will work
duplicated = patients.duplicated(subset=['SSN', 'BIRTHDATE'], keep=False).sum()

if duplicated == 0:
    print('\nThere are no duplicated values when using SSN and BIRTHDATE as patient identifier')
else: 
    print('There are some duplicated values. Investigate!')

There are no missing birthday values.

There are no duplicated values when using SSN and BIRTHDATE as patient identifier


In [8]:
n_patients = len(patients.drop_duplicates(subset=['SSN', 'BIRTHDATE'], keep=False).index)
print("There are a total of {} patients.".format(n_patients))

There are a total of 12352 patients.


----------
**Finding the amount of COVID-19 patients**

Patients are considered COVID-19 patients if they are identified with CODE 840539006.

In total there are 8820 COVID-19 patients. Be aware that it is possible that a single patient has COVID-19 multiple times. Only the amount of occurences for the CODE were counted.



In [9]:
# As can be seen, the CODE column is present in all datasets except the patients dataset.
for k, v in datasets.items():
    print(k.upper())
    print(v.columns)
    print()

CONDITIONS
Index(['START', 'STOP', 'PATIENT', 'ENCOUNTER', 'CODE', 'DESCRIPTION'], dtype='object')

PATIENTS
Index(['Id', 'BIRTHDATE', 'DEATHDATE', 'SSN', 'DRIVERS', 'PASSPORT', 'PREFIX',
       'FIRST', 'LAST', 'SUFFIX', 'MAIDEN', 'MARITAL', 'RACE', 'ETHNICITY',
       'GENDER', 'BIRTHPLACE', 'ADDRESS', 'CITY', 'STATE', 'COUNTY', 'ZIP',
       'LAT', 'LON', 'HEALTHCARE_EXPENSES', 'HEALTHCARE_COVERAGE'],
      dtype='object')

OBSERVATIONS
Index(['DATE', 'PATIENT', 'ENCOUNTER', 'CODE', 'DESCRIPTION', 'VALUE', 'UNITS',
       'TYPE'],
      dtype='object')

CAREPLANS
Index(['Id', 'START', 'STOP', 'PATIENT', 'ENCOUNTER', 'CODE', 'DESCRIPTION',
       'REASONCODE', 'REASONDESCRIPTION'],
      dtype='object')

ENCOUNTERS
Index(['Id', 'START', 'STOP', 'PATIENT', 'ORGANIZATION', 'PROVIDER', 'PAYER',
       'ENCOUNTERCLASS', 'CODE', 'DESCRIPTION', 'BASE_ENCOUNTER_COST',
       'TOTAL_CLAIM_COST', 'PAYER_COVERAGE', 'REASONCODE',
       'REASONDESCRIPTION'],
      dtype='object')



In [10]:
# From the inspection below, the most probable dataset that explains COVID is the conditions dataset.
display_side_by_side(conditions.head(),observations.head(), careplans.head(), encounters.head(), titles=['Conditions','Observations', 'Careplans', 'Encounters'])

Conditions 
 
 
 
 START 
 STOP 
 PATIENT 
 ENCOUNTER 
 CODE 
 DESCRIPTION 
 
 
 
 
 0 
 2019-02-15 
 2019-08-01 
 f0f3bc8d-ef38-49ce-a2bd-dfdda982b271 
 d5ee30a9-362f-429e-a87a-ee38d999b0a5 
 65363002 
 Otitis media 
 
 
 1 
 2019-10-30 
 2020-01-30 
 f0f3bc8d-ef38-49ce-a2bd-dfdda982b271 
 8bca6d8a-ab80-4cbf-8abb-46654235f227 
 65363002 
 Otitis media 
 
 
 2 
 2020-03-01 
 2020-03-30 
 f0f3bc8d-ef38-49ce-a2bd-dfdda982b271 
 681c380b-3c84-4c55-80a6-db3d9ea12fee 
 386661006 
 Fever (finding) 
 
 
 3 
 2020-03-01 
 2020-03-01 
 f0f3bc8d-ef38-49ce-a2bd-dfdda982b271 
 681c380b-3c84-4c55-80a6-db3d9ea12fee 
 840544004 
 Suspected COVID-19 
 
 
 4 
 2020-03-01 
 2020-03-30 
 f0f3bc8d-ef38-49ce-a2bd-dfdda982b271 
 681c380b-3c84-4c55-80a6-db3d9ea12fee 
 840539006 
 COVID-19 
 
 
 Observations 
 
 
 
 DATE 
 PATIENT 
 ENCOUNTER 
 CODE 
 DESCRIPTION 
 VALUE 
 UNITS 
 TYPE 
 
 
 
 
 0 
 2019-08-01 
 f0f3bc8d-ef38-49ce-a2bd-dfdda982b271 
 6a74fdef-2287-44bf-b9e7-18012376faca 
 8302-2 
 Body Height 
 82.7 
 cm 
 numeric 
 
 
 1 
 2019-08-01 
 f0f3bc8d-ef38-49ce-a2bd-dfdda982b271 
 6a74fdef-2287-44bf-b9e7-18012376faca 
 72514-3 
 Pain severity - 0-10 verbal numeric rating [Score] - Reported 
 2.0 
 {score} 
 numeric 
 
 
 2 
 2019-08-01 
 f0f3bc8d-ef38-49ce-a2bd-dfdda982b271 
 6a74fdef-2287-44bf-b9e7-18012376faca 
 29463-7 
 Body Weight 
 12.6 
 kg 
 numeric 
 
 
 3 
 2019-08-01 
 f0f3bc8d-ef38-49ce-a2bd-dfdda982b271 
 6a74fdef-2287-44bf-b9e7-18012376faca 
 77606-2 
 Weight-for-length Per age and sex 
 86.1 
 % 
 numeric 
 
 
 4 
 2019-08-01 
 f0f3bc8d-ef38-49ce-a2bd-dfdda982b271 
 6a74fdef-2287-44bf-b9e7-18012376faca 
 9843-4 
 Head Occipital-frontal circumference 
 46.9 
 cm 
 numeric 
 
 
 Careplans 
 
 
 
 Id 
 START 
 STOP 
 PATIENT 
 ENCOUNTER 
 CODE 
 DESCRIPTION 
 REASONCODE 
 REASONDESCRIPTION 
 
 
 
 
 0 
 fea43343-7312-423f-bb82-b2f5ae71a260 
 2020-03-01 
 2020-03-01 
 f0f3bc8d-ef38-49ce-a2bd-dfdda982b271 
 681c380b-3c84-4c55-80a6-db3d9ea12fee 
 736376001 
 Infectious disease care plan (record artifact) 
 840544004.0 
 Suspected COVID-19 
 
 
 1 
 cbcade35-42bf-4807-8154-3f7f847221e0 
 2020-03-01 
 2020-03-30 
 f0f3bc8d-ef38-49ce-a2bd-dfdda982b271 
 681c380b-3c84-4c55-80a6-db3d9ea12fee 
 736376001 
 Infectious disease care plan (record artifact) 
 840539006.0 
 COVID-19 
 
 
 2 
 51dd78df-2b01-486a-8b33-1fbcd9cec211 
 2020-02-12 
 2020-02-26 
 067318a4-db8f-447f-8b6e-f2f61e9baaa5 
 adedca64-700b-4fb9-82f1-9cbb658abb73 
 91251008 
 Physical therapy procedure 
 44465007.0 
 Sprain of ankle 
 
 
 3 
 8aa5055b-cddc-4170-9e31-e71e5552502a 
 2020-03-13 
 2020-03-13 
 067318a4-db8f-447f-8b6e-f2f61e9baaa5 
 1ea74a77-3ad3-4948-a9cc-3084462035d6 
 736376001 
 Infectious disease care plan (record artifact) 
 840544004.0 
 Suspected COVID-19 
 
 
 4 
 976d369a-2b71-488d-ba20-8674fc272be0 
 2020-03-13 
 2020-04-14 
 067318a4-db8f-447f-8b6e-f2f61e9baaa5 
 1ea74a77-3ad3-4948-a9cc-3084462035d6 
 736376001 
 Infectious disease care plan (record artifact) 
 840539006.0 
 COVID-19 
 
 
 Encounters 
 
 
 
 Id 
 START 
 STOP 
 PATIENT 
 ORGANIZATION 
 PROVIDER 
 PAYER 
 ENCOUNTERCLASS 
 CODE 
 DESCRIPTION 
 BASE_ENCOUNTER_COST 
 TOTAL_CLAIM_COST 
 PAYER_COVERAGE 
 REASONCODE 
 REASONDESCRIPTION 
 
 
 
 
 0 
 d5ee30a9-362f-429e-a87a-ee38d999b0a5 
 2019-02-16T01:02:32Z 
 2019-02-16T01:17:32Z 
 f0f3bc8d-ef38-49ce-a2bd-dfdda982b271 
 5103c940-0c08-392f-95cd-446e0cea042a 
 e2c226c2-3e1e-3d0b-b997-ce9544c10528 
 7c4411ce-02f1-39b5-b9ec-dfbea9ad3c1a 
 outpatient 
 185345009 
 Encounter for symptom 
 129.16 
 129.16 
 69.16 
 65363002.0 
 Otitis media 
 
 
 1 
 6a74fdef-2287-44bf-b9e7-18012376faca 
 2019-08-02T01:02:32Z 
 2019-08-02T01:32:32Z 
 f0f3bc8d-ef38-49ce-a2bd-dfdda982b271 
 0b9f3f7c-8ab6-30a5-b3ae-4dc0e0c00cb3 
 87c33fc5-3fd1-3c52-815a-b89a1623bb3a 
 7c4411ce-02f1-39b5-b9ec-dfbea9ad3c1a 
 wellness 
 410620009 
 Well child visit (procedure) 
 129.16 
 129.16 
 129.16 
 NaN 
 NaN 
 
 
 2 
 8bca6d8a-ab80-4cbf-8abb-46654235f227 
 2019-10-31T01:02:32Z 
 

In [11]:
# Check if indeed the resutls for the code are found in the conditions dataset.
covid_code = 840539006

print("Results:")
for k, v in datasets.items():
    if 'CODE' in v.columns:
        print("{}: {}".format(k.upper(), (v['CODE'] == covid_code).sum()))

Results:
CONDITIONS: 8820
OBSERVATIONS: 0
CAREPLANS: 0
ENCOUNTERS: 0


In [12]:
# As the Conditions dataset tells us how many patients have COVID-19, we use that dataset.
covid_patients = (conditions['CODE'] == covid_code).sum()
print('There are a total of {} COVID-19 patients.'.format(covid_patients))

There are a total of 8820 COVID-19 patients.


-----------
**Finding how many patients have a 'Hospital admission for isolation' encounter.**

There are a total of 1867 hospital admissions for isolation



In [13]:
encounters.head()

,Id,START,STOP,PATIENT,ORGANIZATION,PROVIDER,PAYER,ENCOUNTERCLASS,CODE,DESCRIPTION,BASE_ENCOUNTER_COST,TOTAL_CLAIM_COST,PAYER_COVERAGE,REASONCODE,REASONDESCRIPTION
0,d5ee30a9-362f-429e-a87a-ee38d999b0a5,2019-02-16T01:02:32Z,2019-02-16T01:17:32Z,f0f3bc8d-ef38-49ce-a2bd-dfdda982b271,5103c940-0c08-392f-95cd-446e0cea042a,e2c226c2-3e1e-3d0b-b997-ce9544c10528,7c4411ce-02f1-39b5-b9ec-dfbea9ad3c1a,outpatient,185345009,Encounter for symptom,129.16,129.16,69.16,65363002.0,Otitis media
1,6a74fdef-2287-44bf-b9e7-18012376faca,2019-08-02T01:02:32Z,2019-08-02T01:32:32Z,f0f3bc8d-ef38-49ce-a2bd-dfdda982b271,0b9f3f7c-8ab6-30a5-b3ae-4dc0e0c00cb3,87c33fc5-3fd1-3c52-815a-b89a1623bb3a,7c4411ce-02f1-39b5-b9ec-dfbea9ad3c1a,wellness,410620009,Well child visit (procedure),129.16,129.16,129.16,NaN,NaN
2,8bca6d8a-ab80-4cbf-8abb-46654235f227,2019-10-31T01:02:32Z,2019-10-31T01:17:32Z,f0f3bc8d-ef38-49ce-a2bd-dfdda982b271,5103c940-0c08-392f-95cd-446e0cea042a,e2c226c2-3e1e-3d0b-b997-ce9544c10528,7c4411ce-02f1-39b5-b9ec-dfbea9ad3c1a,outpatient,185345009,Encounter for symptom,129.16,129.16,69.16,65363002.0,Otitis media
3,821e57ac-9304-46a9-9f9b-83daf60e9e43,2020-01-31T01:02:32Z,2020-01-31T01:17:32Z,f0f3bc8d-ef38-49ce-a2bd-dfdda982b271,0b9f3f7c-8ab6-30a5-b3ae-4dc0e0c00cb3,87c33fc5-3fd1-3c52-815a-b89a1623bb3a,7c4411ce-02f1-39b5-b9ec-dfbea9ad3c1a,wellness,410620009,Well child visit (procedure),129.16,129.16,129.16,NaN,NaN
4,681c380b-3c84-4c55-80a6-db3d9ea12fee,2020-03-02T01:02:32Z,2020-03-02T01:58:32Z,f0f3bc8d-ef38-49ce-a2bd-dfdda982b271,fd328395-ab1d-35c6-a2d0-d05a9a79cf11,9c875a09-93e0-39aa-9260-ad264bbdd3fe,7c4411ce-02f1-39b5-b9ec-dfbea9ad3c1a,ambulatory,185345009,Encounter for symptom (procedure),129.16,129.16,69.16,NaN,NaN


In [14]:
# Investigate the DESCRIPTION column
sorted(encounters['DESCRIPTION'].unique())

['Admission to intensive care unit (procedure)',
 'Admission to surgical department',
 'Admission to thoracic surgery department',
 'Allergic disorder follow-up assessment',
 'Allergic disorder initial assessment',
 'Asthma follow-up',
 'Cardiac Arrest',
 'Consultation for treatment',
 'Death Certification',
 'Diagnosis of cystic fibrosis using sweat test and gene test',
 'Discussion about treatment (procedure)',
 'Domiciliary or rest home patient evaluation and management',
 'Drug rehabilitation and detoxification',
 'Emergency Encounter',
 'Emergency Room Admission',
 'Emergency hospital admission for asthma',
 'Emergency room admission (procedure)',
 'Encounter Inpatient',
 "Encounter for 'check-up'",
 'Encounter for Problem',
 'Encounter for check up',
 'Encounter for check up (procedure)',
 'Encounter for problem',
 'Encounter for problem (procedure)',
 'Encounter for symptom',
 'Encounter for symptom (procedure)',
 'Follow-up encounter',
 'Follow-up visit (procedure)',
 'General 

In [15]:
# Find the amount of patients with Hospital admission for isolation (procedure)
string = 'Hospital admission for isolation (procedure)'
n_admission_isolation = (encounters['DESCRIPTION'] == string).sum()
print('There are a total of {} hospital admissions for isolation'.format(n_admission_isolation))

There are a total of 1867 hospital admissions for isolation


--------------------------

**Find out how many people passed away.**

Patients that did not survived Covid have a `DEATHDATE` which is not null. 

A total of 2352 patients died.


In [16]:
# because every Id (which is unique for every row) represents a unique patient indirectly,
# the step to select all patients based on SSN and BIRTHDATE is skipped.
n_died = patients['DEATHDATE'].notna().sum()
print('A total of {} patients died.'.format(n_died))

A total of 2352 patients died.


### 1.2 Test your solution
The following function needs to be called. You can use this as a test. There are however more meaningful overviews 
you can create. 

In [17]:
def part1(num_pat, num_cov, num_admitted, num_died):
    print(f'There are {num_pat} patients in total')
    print(f'There are {num_cov} covid patients')
    print(f'There are {num_admitted} admitted patients')
    print(f'{num_died} patients died')

part1(n_patients, covid_patients, n_admission_isolation, n_died)

There are 12352 patients in total
There are 8820 covid patients
There are 1867 admitted patients
2352 patients died


### Expected outcome

---

<a name='1'></a>
## Part 2: Data Wrangling: set up the dataframe (30 pt)

In this part we are going to combine data to create a dataframe with values of interest for the lab values analysis. 

We would like a dataframe containing the following information per record (only Covid patients!!!)

- `PATIENT` - the ID of the covid patient
- `days` - the number of days the patient is under observation
- `CODE-Y` - the code of the observation  
- `VALUE` - the lab value of the observation

where only the following observation codes needs to be selected:

- `48065-7`  Fibrin D-dimer FEU [Mass/volume] in Platelet poor plasma
- `26881-3`   Interleukin 6 [Mass/volume] in Serum or Plasma
- `2276-4` Ferritin [Mass/volume] in Serum or Plasma
- `89579-7` Troponin I.cardiac [Mass/volume] in Serum or Plasma by High sensitivity method
- `731-0` Lymphocytes [#/volume] in Blood by Automated count
- `14804-9` Lactate dehydrogenase [Enzymatic activity/volume] in Serum or Plasma by Lactate to pyruvate reaction

The days information is not primarely available and needs to be calculated by substracting observation DATE - START. 

An example of such a dataframe is given below:

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Hints</b></font>
</summary>
    <ul><li>you can use pandas.DataFrame.merge() to merge dataframes</li>
    <li>df = df[(df.CODE == condition1 | df.CODE == condition1 )] selects rows with CODE of 2 conditional values</li>
    <li>df.DATE - df.START return days if DATE and START are datetime format</li>
    <li>pd.to_datetime() can be used to typecast to datetime</li>
</ul>
</details>

<a name='ex-21'></a>
### 2.1 Code your solution


    PATIENT - the ID of the covid patient
    days - the number of days the patient is under observation
    CODE-Y - the code of the observation
    VALUE - the lab value of the observation


In [18]:
# concatenate observations and conditions
# day = start - date
columns_pat = [
    'Id',
]
columns_cond = [
    'START',
    'PATIENT',
    'CONDITION_CODE'
]

columns_obs = [
    'PATIENT',
    'DATE',
    'CODE',
    'VALUE',
    'UNITS'
]

patients = patients[patients.columns.intersection(columns_pat)].set_index('Id')

covid_patients = conditions[conditions['CODE'] == covid_code].rename(columns={'CODE':'CONDITION_CODE'})
covid_patients = covid_patients[covid_patients.columns.intersection(columns_cond)].set_index('PATIENT')

observations = observations[observations.columns.intersection(columns_obs)].set_index('PATIENT')

covid_patients = covid_patients.merge(observations, left_index=True, right_index=True)
covid_patients.head()

,START,CONDITION_CODE,DATE,CODE,VALUE,UNITS
PATIENT,,,,,,
0000b247-1def-417a-a783-41c8682be022,2020-02-18,840539006,2019-12-23,8302-2,146.0,cm
0000b247-1def-417a-a783-41c8682be022,2020-02-18,840539006,2019-12-23,72514-3,1.0,{score}
0000b247-1def-417a-a783-41c8682be022,2020-02-18,840539006,2019-12-23,29463-7,37.5,kg
0000b247-1def-417a-a783-41c8682be022,2020-02-18,840539006,2019-12-23,39156-5,17.6,kg/m2
0000b247-1def-417a-a783-41c8682be022,2020-02-18,840539006,2019-12-23,59576-9,42.3,%


In [19]:
covid_patients.dtypes

START             object
CONDITION_CODE     int64
DATE              object
CODE              object
VALUE             object
UNITS             object
dtype: object

In [20]:
covid_patients = covid_patients.astype({
    'START': 'datetime64[ns]',
    'DATE': 'datetime64[ns]',
})

covid_patients['DAYS'] = covid_patients['DATE'] - covid_patients['START']

In [21]:
to_include = [
    '48065-7',
    '26881-3',
    '2276-4',
    '89579-7',
    '731-0',
    '14804-9'
]

filtered_patients = covid_patients[covid_patients['CODE'].isin(to_include)]
filtered_patients = filtered_patients.rename(columns={'CODE': 'CODE-Y'})

In [22]:
filtered_patients = filtered_patients.astype({'VALUE':'float'})

In [23]:
filtered_patients.head()

,START,CONDITION_CODE,DATE,CODE-Y,VALUE,UNITS,DAYS
PATIENT,,,,,,,
00079a57-24a8-430f-b4f8-a1cf34f90060,2020-02-25,840539006,2020-02-25,731-0,1.1,10*3/uL,0 days
00079a57-24a8-430f-b4f8-a1cf34f90060,2020-02-25,840539006,2020-02-25,48065-7,0.4,ug/mL,0 days
00079a57-24a8-430f-b4f8-a1cf34f90060,2020-02-25,840539006,2020-02-25,2276-4,489.5,ug/L,0 days
00079a57-24a8-430f-b4f8-a1cf34f90060,2020-02-25,840539006,2020-02-25,89579-7,3.4,pg/mL,0 days
00079a57-24a8-430f-b4f8-a1cf34f90060,2020-02-25,840539006,2020-02-25,14804-9,241.8,U/L,0 days


---

<a name='2'></a>
## Part 3: Data Wrangling, split into survived and not survived (10 pt)

Now we have the required data we would like to split the data into survived and not survived. First we fetch all the ids of the survived and deceased patients. We can use these ids to select the records of the survived patients and the patients that did not survived

Your job is to split the data into survived and not survived records. There are multiple ways to do this. One way is the  `.isin()` method

In [24]:
# reloading because I used different variables

base_path = '../data/covid/'

conditions = pd.read_csv(base_path + 'conditions.csv')
patients = pd.read_csv(base_path + 'patients.csv')
observations = pd.read_csv(base_path + 'observations.csv')
care_plans = pd.read_csv(base_path + 'careplans.csv')
encounters = pd.read_csv(base_path + 'encounters.csv')

In [25]:
#the following code is given, RUN THIS CELL
#get survived and deceased ids
completed_isolation_patients = care_plans[(care_plans.CODE == 736376001) & (care_plans.STOP.notna()) \
                                          & (care_plans.REASONCODE == 840539006)].PATIENT
negative_covid_patient_ids = observations[(observations.CODE == '94531-1') \
                                          & (observations.VALUE == 'Not detected (qualifier value)')].PATIENT.unique()
survivor_ids = np.union1d(completed_isolation_patients, negative_covid_patient_ids)
deceased_ids = patients[patients.DEATHDATE.notna()].Id

<a name='ex-31'></a>
### 3.1 Code your solution

In [26]:
survivors = filtered_patients[filtered_patients.index.isin(survivor_ids)]
deceased = filtered_patients[filtered_patients.index.isin(deceased_ids)]

n_survivors = len(survivors)
n_deceased = len(deceased)

### 3.2 Test your solution

In [27]:
def test3(survived, died):
    print(f'patients records survived: {survived}, patients records deceased {died}')

test3(n_survivors, n_deceased)

patients records survived: 57303, patients records deceased 16793


#### Expected outcome

---

<a name='3'></a>
## Part 4: Plot the data (20 pt)

Create plots with the lab data, for each code one plot. Separate the survivors and the deceased by color. An example of such a plot is given below. You can create 6 plots in one grid (for each code one plot) or use a widget (for instance a drop down menu widget) to select a lab CODE. Plot on the x-axis the days, on the y-axis the VALUE. Use proper labels, titles and legends.

<img src="../images/week3_plot.png" width="500" height="500"/>

<a name='ex-41'></a>
### 4.1 Code your solution

In [28]:
from bokeh.models import ColumnDataSource, CustomJS, Range1d, Select, Legend
from bokeh.plotting import figure, show, output_notebook,gridplot
from bokeh.layouts import column, gridplot

output_notebook()

Loading BokehJS ...

In [29]:
codes = filtered_patients['CODE-Y'].unique()
print('Codes available: {}'.format(codes))

Codes available: ['731-0' '48065-7' '2276-4' '89579-7' '14804-9' '26881-3']


In [30]:
# Create 6 plots in a grid, 2x3
grid_frames = []

for code in codes:
    frame_s = survivors[survivors['CODE-Y'] == code]
    frame_d = deceased[deceased['CODE-Y'] == code]
    
    # check if units are the same
    units_s = frame_s['UNITS'].unique()
    units_d = frame_d['UNITS'].unique()
    
    if units_s != units_d:
        raise('Measurement units are not the same!')
    elif len(units_s) > 1 or len(units_d) > 1:
        raise('Expects one measurement unit!')
    
    ps = figure(title="Patients code {}".format(code), background_fill_color="#f0f0f0", x_axis_label='days', y_axis_label=units_s[0])
    s = ps.scatter(frame_s['DAYS'].dt.days, frame_s['VALUE'], color='lightblue')
    d = ps.scatter(frame_d['DAYS'].dt.days, frame_d['VALUE'], color='red')
    
    legend = Legend(items=[
    ("Survivors"   , [s]),
    ("Deceased" , [d])], location="center")

    ps.add_layout(legend, 'below')
    
    grid_frames.append(ps)

In [31]:
grid_frames = np.array(grid_frames).reshape((2, 3))
grid = gridplot(grid_frames.tolist(), plot_width=400, plot_height=400)
show(grid)

<a name='4'></a>
## Part 5: Plot the location of the patients (10 pt)

This is a bonus part. Can you plot the patients location on a map? See also 
https://docs.bokeh.org/en/latest/docs/user_guide/geo.html


<a name='ex-51'></a>
### 5.1 Code your solution

In [32]:
def merc_from_arrays(lats, lons):
    """
    https://gis.stackexchange.com/questions/156035/calculating-mercator-coordinates-from-lat-lon
    """
    r_major = 6378137.000
    x = r_major * np.radians(lons)
    scale = x/lons
    y = 180.0/np.pi * np.log(np.tan(np.pi/4.0 + lats * (np.pi/180.0)/2.0)) * scale
    return (x, y)

x, y = merc_from_arrays(patients['LAT'], patients['LON'])

In [33]:
coords_df = pd.DataFrame({'x': x, 'y': y})

In [34]:
from bokeh.plotting import figure, show
from bokeh.tile_providers import CARTODBPOSITRON, get_provider

tile_provider = get_provider(CARTODBPOSITRON)

# range bounds supplied in web mercator coordinates
p = figure(x_range=(-8500000, -7500000), y_range=(5190000, 5200000),
           x_axis_type="mercator", y_axis_type="mercator")
p.add_tile(tile_provider)

p.scatter(x=coords_df['x'].tolist(), y=coords_df['y'].tolist())

show(p)
